<a href="https://colab.research.google.com/github/jamesETsmith/2022_simons_collab_pyscf_workshop/blob/main/demos/05_Excited_States.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up the Jupyter notebook

* We need to install a few things before we get started
  * [PySCF](https://pyscf.org/) (for the quantum chemsitry)
  * [py3DMol](https://3dmol.csb.pitt.edu/) for visualizing the molecule
  * [plotly](https://plotly.com/python/) and kaleido for plotting

In [7]:
%pip install -q pyscf py3DMol plotly kaleido pandas

# Chemical System Setup

In this example, we're going to study the electronic excitations of 1,2-oxazole with [time-dependent density functional theory (TD-DFT)](https://en.wikipedia.org/wiki/Time-dependent_density_functional_theory#:~:text=Time%2Ddependent%20density%2Dfunctional%20theory,as%20electric%20or%20magnetic%20fields.).
We point out that just like the other examples, we cut corners (e.g. using a small basis set) to make these calculations run quicker on Google Colab.
For research 

In [16]:
from pyscf import gto, scf, dft, tddft
import py3Dmol

In [37]:
mol = gto.M(atom="""  C      1.0701      0.4341     -0.0336
  C      0.8115     -0.9049     -0.1725
  C     -0.6249     -1.0279     -0.0726
  H      1.9842      1.0231     -0.0364
  H      1.5156     -1.7176     -0.3255
  H     -1.2289     -1.9326     -0.1322
  O     -0.0873      1.1351      0.1422
  N     -1.1414      0.1776      0.1122""",
  # basis = "631g", 
  verbose=3)

xyz_view = py3Dmol.view(width=400,height=400)
xyz_view.addModel(mol.tostring(format="xyz"),'xyz')
xyz_view.setStyle({'stick':{}, "sphere":{"radius":0.4}})
xyz_view.setBackgroundColor('0xeeeeee')
xyz_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Helper Functions

Here we create helper functions to run DFT, TD-DFT, and perform the spectral analysis. Note that a gaussian distribution is a rough way to approximate the line shape for each excitation.

In [38]:
import numpy as np
from scipy.constants import physical_constants

ha_2_ev = 1/physical_constants["electron volt-hartree relationship"][0]

def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

def run_spectral_analysis(mol, xc="b3lyp"):
  n_states=15
  spectral_width=0.1

  # Ground State DFT
  mf = dft.RKS(mol, xc=xc).run()

  # Excited State DFT
  mytd = tddft.TDDFT(mf)
  mytd.nstates = n_states
  mytd.kernel();
  mytd.analyze()
  osc_strengths = mytd.oscillator_strength()[:n_states-5]

  # Convolve lineshapes to make spectra
  energies_ev = mytd.e[:n_states-5]*ha_2_ev
  x_range = np.linspace(0.0, energies_ev.max()*1.1, num=1000)
  intensity = np.zeros(x_range.size)

  for e, f in zip(energies_ev, osc_strengths):
    intensity += gaussian(x_range, e, spectral_width) * f

  # Rough Normalization
  dx = (x_range[-1] - x_range[0])/x_range.size
  area = (intensity*dx).sum()
  intensity /= area


  return x_range, intensity

In [39]:
import pandas as pd

data = {"Excitation Energy (eV)":[], "Intensity":[], "Exchange-Correlation Functional":[]}

xcs = ["lda", "pbe"] # Should run in < 2 mins
xcs = ["lda", "pbe", "camb3lyp"] # May take a couple of minutes!

for xc in xcs:
  x_range, intensity = run_spectral_analysis(mol, xc=xc)

  data["Excitation Energy (eV)"] += x_range.tolist()
  data["Intensity"] += intensity.tolist()
  data["Exchange-Correlation Functional"] += [xc]*x_range.size

df = pd.DataFrame(data)


converged SCF energy = -238.456268099995
Excited State energies (eV)
[ 5.85858931  7.17536147  7.55773107  7.95623303  8.52179116  9.04484151
  9.2394843   9.72029867 10.00409261 10.0929612  10.18533752 10.26885227
 11.05679332 11.45784061 12.02415608]

** Singlet excitation energies and oscillator strengths **
Excited State   1:      5.85859 eV    211.63 nm  f=0.0064
Excited State   2:      7.17536 eV    172.79 nm  f=0.0356
Excited State   3:      7.55773 eV    164.05 nm  f=0.0090
Excited State   4:      7.95623 eV    155.83 nm  f=0.0367
Excited State   5:      8.52179 eV    145.49 nm  f=0.0043
Excited State   6:      9.04484 eV    137.08 nm  f=0.0010
Excited State   7:      9.23948 eV    134.19 nm  f=0.0360
Excited State   8:      9.72030 eV    127.55 nm  f=0.0782
Excited State   9:     10.00409 eV    123.93 nm  f=0.0017
Excited State  10:     10.09296 eV    122.84 nm  f=0.0001
Excited State  11:     10.18534 eV    121.73 nm  f=0.2026
Excited State  12:     10.26885 eV    120.74 nm  

# Plot the UV/Vis Spectra

Since we've stored our data in a dataframe, we can plot it quickly using `plotly.express`.

In [40]:
import plotly.express as px
fig = px.line(df, x="Excitation Energy (eV)", y="Intensity", markers=True, color="Exchange-Correlation Functional")
fig.show()